In [ ]:
#pip install --upgrade google-api-python-client
#pip install --upgrade google-cloud-storage
#pip install --upgrade google-cloud-spanner

In [18]:
import pandas as pd
import ibm_db_dbi as idb
from sqlalchemy import create_engine
import pyodbc
import warnings
import pyvalidata as pvd
import hashlib
#import time 
#import datetime
from datetime import datetime
import numpy as np
import sqlalchemy
import ibm_db_sa
import csv
from google.cloud import storage
from google.cloud import spanner
import os
import pandas as pd
from google.cloud import spanner

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="kr-7738-supvis-t-3e70a14d235d_test.json"

In [5]:
#projects/kr-7738-supvis-t/instances/nc-event-sink-instance/databases/storewalks-db

# Your Cloud Spanner instance ID.
instance_id = "nc-event-sink-instance"

# Your Cloud Spanner database ID.
database_id_1 = "udm-db"
database_id_2 = "storewalks-db"

# Instantiate a client.
spanner_client = spanner.Client()

# Get a Cloud Spanner instance by ID.
instance = spanner_client.instance(instance_id)

# Get a Cloud Spanner database by ID (for database_id_1).
database_1 = instance.database(database_id_1)
print("Connection to udm-db (database 1) successful.")

# Get a Cloud Spanner database by ID (for database_id_1).
database_2 = instance.database(database_id_2)
print("Connection to storewalks-db (database 2) successful.")

Connection to udm-db (database 1) successful.
Connection to storewalks-db (database 2) successful.


## 1) Daily Counts & Data Validation between source and target ##

In [6]:
#database_id_1 = "udm-db"  & instance_id = "nc-event-sink-instance" (Source Query)

#Source Data
with database_1.snapshot() as snapshot:
    result = snapshot.execute_sql("""
        SELECT * FROM (
            SELECT
                div_str_id AS div_str_id_s,
                task_type_name,
                task_classification AS task_classification_s,
                a.task_desc,
            FROM
                (SELECT * FROM santn_rotn_task_mstr) a
                JOIN (SELECT * FROM sub_dpmt_mpng) b USING (sub_dpmt_id)
                JOIN (SELECT * FROM str_dpmt_mpng) c ON c.dpmt_name = b.prmry_dpmt_name
                LEFT JOIN (SELECT * FROM rolng_task_addtnl_info) d
                    ON (a.santn_rotn_task_mstr_id = d.santn_rotn_task_mstr_id)
            WHERE
                a.is_actv = 'Y'
                AND c.is_actv = 'Y'
                AND DATE(strt_date) = '2023-09-05'
                --AND strt_date <= '2023-09-05'
                AND task_classification LIKE 'Daily'
                AND div_str_id = '53100515'
            UNION ALL
            SELECT
                div_str_id AS div_str_id_s,
                task_type_name,
                task_classification AS task_classification_s,
                a.task_desc,
            FROM
                (SELECT * FROM santn_rotn_task_mstr) a
                JOIN (SELECT * FROM sub_dpmt_mpng) b USING (sub_dpmt_id)
                JOIN (SELECT * FROM str_dpmt_mpng) c ON c.dpmt_name = b.prmry_dpmt_name
                LEFT JOIN (SELECT * FROM santn_rotn_task_schdl) d
                    ON (a.santn_rotn_task_mstr_id = d.santn_rotn_task_mstr_id)
            WHERE
                a.is_actv = 'Y'
                AND c.is_actv = 'Y'
                AND DATE(schd_strt_date) = '2023-09-05'
                --AND schd_strt_date <= '2023-09-05'
                AND task_classification LIKE 'Daily'
                AND div_str_id = '53100515'
        ) Daily_S
    """)
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_s_df = pd.DataFrame(rows, columns = cols)

In [7]:
num_rows, num_columns = result_s_df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
result_s_df.head()

Number of rows: 44
Number of columns: 4


,div_str_id_s,task_type_name,task_classification_s,task_desc
0,53100515,Sanitation,Daily,Clean/Sweep - Work Area
1,53100515,Sanitation,Daily,Wipe Down - Prep Area Work Tables and Scales
2,53100515,Sanitation,Daily,Take Out Trash
3,53100515,Sanitation,Daily,Sweep Pick-up Cooler Floor
4,53100515,Sanitation,Daily,Vestibule Maintenance


In [8]:
#database_id_2 = "storewalks-db"  & instance_id = "nc-event-sink-instance"
#Target Data
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
        SELECT
            CONCAT(division_number, store_number) AS div_str_id,
            task_type,
            task_classification,
            task_description,
        FROM
            task
        WHERE
            task_type <> 'Green Rack'
            AND task_classification LIKE 'Daily'
            AND task_date = '2023-09-05'
            AND CONCAT(division_number, store_number) = '53100515'
    """)
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_t_df = pd.DataFrame(rows, columns=cols)


In [9]:
num_rows, num_columns = result_t_df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
result_t_df.head()

Number of rows: 44
Number of columns: 4


,div_str_id,task_type,task_classification,task_description
0,53100515,Sanitation,Daily,Wipe Down - Glass Doors on Sales Floor Display...
1,53100515,Sanitation,Daily,Clean - Cutting Room 3-compt Sink
2,53100515,Sanitation,Daily,Spray and Wipe - Prep Room Weigh Scale
3,53100515,Sanitation,Daily,Clean - Parking Lot and Sidewalk
4,53100515,Sanitation,Daily,Spray and Wipe - Food Preparation Tables


In [51]:
# Merge the two DataFrames on 'div_str_id' and 'task_desc'
merged_df = pd.merge(result_s_df, result_t_df, left_on=['div_str_id_s', 'task_desc'], right_on=['div_str_id', 'task_description'], how='outer')

# Create a new column 'ComparisonResult' based on your conditions
merged_df['ComparisonResult'] = 'Mismatch'

# Condition 1: Matched
condition_matched = (
    (merged_df['task_type_name'] == merged_df['task_type']) &
    (merged_df['task_classification_s'] == merged_df['task_classification']) &
    #(merged_df['sub_dpmt_name'] == merged_df['department_name']) &
    (merged_df['task_desc'] == merged_df['task_description'])
    #(merged_df['concatenated_s'] == merged_df['concatenated_t'])
)
merged_df.loc[condition_matched, 'ComparisonResult'] = 'Matched'

# Condition 2: Record NOT in Target
condition_not_in_target = (merged_df['div_str_id_s'].notna()) & (merged_df['div_str_id'].isna())
merged_df.loc[condition_not_in_target, 'ComparisonResult'] = 'Record NOT in Target'

# Condition 3: Record NOT in Source
condition_not_in_source = (merged_df['div_str_id'].notna()) & (merged_df['div_str_id_s'].isna())
merged_df.loc[condition_not_in_source, 'ComparisonResult'] = 'Record NOT in Source'

# Remove duplicates from merged_df
merged_df = merged_df.drop_duplicates()

# Reset the index after removing duplicates (optional)
merged_df.reset_index(drop=True, inplace=True)

# Check if there are any failed records
if not failed_records.empty:
    print('Daily Validation Report Failed')

    # Get the current date as a timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    # Define the full path to save the report file
    file_path = r'C:\\Users\\AS81970\\Desktop\\2023 Q3\\Store Walk-in\\Reports'
    
    # Define the file name with the date appended
    report_file_name = f'Daily_Validation_Report_Failed_{timestamp}.xlsx'
    
    # Save the failed records to an Excel file with the generated file name
    failed_records.to_excel(f'{file_path}\\{report_file_name}', index=False)
else:
    print('Daily Validation Report Passed')  

Daily Validation Report Passed


In [ ]:
#counts

In [39]:
#Source Counts
with database_1.snapshot() as snapshot:
    result = snapshot.execute_sql("""
 select sum(count), 'daily_s'  
from    
 (
 (select  distinct count(*) as count,div_str_id, task_type_name, task_classification,sub_dpmt_name 
            FROM
                (SELECT * FROM santn_rotn_task_mstr) a
                JOIN (SELECT * FROM sub_dpmt_mpng) b USING (sub_dpmt_id)
                JOIN (SELECT * FROM str_dpmt_mpng) c ON c.dpmt_name = b.prmry_dpmt_name
                LEFT JOIN (SELECT * FROM rolng_task_addtnl_info) d
                    ON (a.santn_rotn_task_mstr_id = d.santn_rotn_task_mstr_id)
            WHERE
                a.is_actv = 'Y'
                AND c.is_actv = 'Y'
                AND DATE(strt_date) = '2023-09-05'
                --AND strt_date <= '2023-09-05'
                AND task_classification LIKE 'Daily'
group by div_str_id, task_type_name, task_classification,sub_dpmt_name)  
union all
(select  distinct count(*) as count,div_str_id, task_type_name, task_classification,sub_dpmt_name 
            FROM
                (SELECT * FROM santn_rotn_task_mstr) a
                JOIN (SELECT * FROM sub_dpmt_mpng) b USING (sub_dpmt_id)
                JOIN (SELECT * FROM str_dpmt_mpng) c ON c.dpmt_name = b.prmry_dpmt_name
                LEFT JOIN (SELECT * FROM santn_rotn_task_schdl) d
                    ON (a.santn_rotn_task_mstr_id = d.santn_rotn_task_mstr_id)
            WHERE
                a.is_actv = 'Y'
                AND c.is_actv = 'Y'
                AND DATE(schd_strt_date) = '2023-09-05'
                --AND schd_strt_date <= '2023-09-05'
                AND task_classification LIKE 'Daily'
group by div_str_id, task_type_name, task_classification,sub_dpmt_name) 
)   
    
        """)
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_s_c_df = pd.DataFrame(rows, columns = cols)

In [40]:
result_s_c_df.head()

,,
0,109079,daily_s


In [54]:
#Source Counts
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select sum(count1), 'daily_t' 
from 
(select count(*) as count1,division_number, store_number, task_type, task_classification,department_name, task_date, task_status 
from  task
where task_type<>'Green Rack' and task_classification like 'Daily' and task_date='2023-09-05'
group by division_number, store_number, task_type, task_classification,department_name, task_date, task_status
)
        """)
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_t_c_df = pd.DataFrame(rows, columns = cols)

In [55]:
result_t_c_df.head()

,,
0,108989,daily_t


In [58]:
# Get the current timestamp
timestamp_c = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Define the file path
file_path_c = f"C:\\Users\\AS81970\\Desktop\\2023 Q3\\Store Walk-in\\Reports\\Count_{timestamp_c}.csv"

# Write the DataFrames to the CSV file
result_s_c_df.to_csv(file_path_c, index=False)
result_t_c_df.to_csv(file_path_c, mode='a', header=False, index=False)

## 2) Weekly Sanitation Counts & Data Validation between source and target ##

In [71]:
#source data1
with database_1.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select * from
(
select div_str_id,a.task_desc,a.task_type, task_type_name, task_classification,sub_dpmt_name
from (SELECT * FROM santn_rotn_Task_mstr) a 
join (SELECT * FROM sub_dpmt_mpng) b using(sub_dpmt_id) 
join (SELECT * FROM str_dpmt_mpng) c on c.dpmt_name=b.prmry_dpmt_name 
left join (SELECT * FROM rolng_task_addtnl_info)  d on (a.santn_rotn_task_mstr_id=d.santn_rotn_task_mstr_id)  
where a.is_actv='Y' and c.is_actv='Y' and DATE(strt_date) >='2023-08-30' and DATE(strt_date) <='2023-09-05' and task_classification  like 'Weekly' and div_str_id ='53100515'
union all 
select div_str_id,a.task_desc,a.task_type, task_type_name, task_classification,sub_dpmt_name
from (SELECT * FROM santn_rotn_Task_mstr) a 
join (SELECT * FROM sub_dpmt_mpng) b using(sub_dpmt_id) 
join (SELECT * FROM str_dpmt_mpng) c on c.dpmt_name=b.prmry_dpmt_name 
left join (SELECT * FROM santn_rotn_task_schdl)  d on (a.santn_rotn_task_mstr_id=d.santn_rotn_task_mstr_id) 
where a.is_actv='Y' and c.is_actv='Y' and DATE(schd_strt_date) >='2023-08-30' and DATE(schd_strt_date) <='2023-09-05' and task_classification  like 'Weekly' and div_str_id ='53100515'
) 

 """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_ws_s1_df = pd.DataFrame(rows, columns = cols)    

In [92]:
#Source data2
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select 
concat(division_number,store_number) AS div_str_id,
task_type, task_classification, department_name, task_description
from task
where task_type='Sanitation' 
and task_classification like 'Weekly' 
and task_date='2023-09-04' 
and task_status='To Do'
        """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_ws_s2_df = pd.DataFrame(rows, columns = cols)    

In [93]:
# Joining the DataFrames using the specified conditions
result_ws_s_df = pd.merge(result_ws_s1_df, result_ws_s2_df, how='inner', left_on=[
    'div_str_id', 'task_type_name', 'task_classification','sub_dpmt_name', 'task_desc'
], right_on=[
    'div_str_id', 'task_type', 'task_classification', 'department_name', 'task_description'
])

# Select the desired columns in the result_df DataFrame
result_ws_s_df = result_ws_s_df[['div_str_id', 'task_desc', 'task_type_x', 'task_type_name', 'task_classification', 'sub_dpmt_name']]

# Rename the columns in the result_df DataFrame
result_ws_s_df = result_ws_s_df.rename(columns={
    'div_str_id': 'div_str_id_s',
    'task_type_x': 'task_type_name',
    'task_classification': 'task_classification_s',
    'task_desc': 'task_desc',
    'sub_dpmt_name': 'sub_dpmt_name'
})

In [94]:
result_ws_s_df.head()

,div_str_id_s,task_desc,task_type_name,task_type_name,task_classification_s,sub_dpmt_name
0,53100515,Deep Clean - Knife Racks,Weekly Sanitation,Sanitation,Weekly,Meat
1,53100515,Wash/Rinse/Sanitize - Produce Tubs,Weekly Sanitation,Sanitation,Weekly,Produce
2,53100515,Deep Clean - Floor Drains,Weekly Sanitation,Sanitation,Weekly,Bakery
3,53100515,Spray and Wipe - Wipe Down Walls Behind Cut Fr...,Weekly Sanitation,Sanitation,Weekly,Produce
4,53100515,Spray and Wipe - Bottom Pan of Refrigeration C...,Weekly Sanitation,Sanitation,Weekly,Floral


In [83]:
#result_ws_s_df.to_csv("C:\\Users\\AS81970\\Desktop\\tm_ws_s.csv", index=False)

In [87]:
#Target data
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select 
concat(division_number,store_number) AS div_str_id,
task_type, task_classification, department_name, task_description
from task
where task_type<>'Green Rack' 
and task_classification  like 'Weekly'
and task_date='2023-09-05'
and task_type='Sanitation'
and concat(division_number, store_number) = '53100515'
        """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_ws_t_df = pd.DataFrame(rows, columns = cols)    

In [88]:
result_ws_t_df.head()

,div_str_id,task_type,task_classification,department_name,task_description
0,53100515,Sanitation,Weekly,Meat,Deep Clean - Floor Drains
1,53100515,Sanitation,Weekly,Produce,Spray and Wipe - Refrigerated Island Cases inc...
2,53100515,Sanitation,Weekly,Frozen,Clean - Glass Doors
3,53100515,Sanitation,Weekly,Floral,"Spray and Wipe - Floral Display Case Doors, Ha..."
4,53100515,Sanitation,Weekly,Meat,"Spot Clean - Beef, Pork, & Poultry Display Cas..."


In [95]:
# Merge the two DataFrames on 'div_str_id' and 'task_desc'
ws_merged_df = pd.merge(result_s_df, result_t_df, left_on=['div_str_id_s', 'task_desc'], right_on=['div_str_id', 'task_description'], how='outer')

# Create a new column 'ComparisonResult' based on your conditions
ws_merged_df['ComparisonResult'] = 'Mismatch'

# Condition 1: Matched
condition_matched = (
    (ws_merged_df['task_type_name'] == ws_merged_df['task_type']) &
    (ws_merged_df['task_classification_s'] == ws_merged_df['task_classification']) &
    #(ws_merged_df['sub_dpmt_name'] == ws_merged_df['department_name']) &
    (ws_merged_df['task_desc'] == ws_merged_df['task_description'])
    #(ws_merged_df['concatenated_s'] == ws_merged_df['concatenated_t'])
)
ws_merged_df.loc[condition_matched, 'ComparisonResult'] = 'Matched'

# Condition 2: Record NOT in Target
condition_not_in_target = (ws_merged_df['div_str_id_s'].notna()) & (ws_merged_df['div_str_id'].isna())
ws_merged_df.loc[condition_not_in_target, 'ComparisonResult'] = 'Record NOT in Target'

# Condition 3: Record NOT in Source
condition_not_in_source = (ws_merged_df['div_str_id'].notna()) & (ws_merged_df['div_str_id_s'].isna())
ws_merged_df.loc[condition_not_in_source, 'ComparisonResult'] = 'Record NOT in Source'

# Remove duplicates from ws_merged_df
ws_merged_df = ws_merged_df.drop_duplicates()

# Reset the index after removing duplicates (optional)
ws_merged_df.reset_index(drop=True, inplace=True)

# Check if there are any failed records
if not failed_records.empty:
    print('Weekly Sanitation Validation Report Failed')

    # Get the current date as a timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    # Define the full path to save the report file
    file_path = r'C:\\Users\\AS81970\\Desktop\\2023 Q3\\Store Walk-in\\Reports'
    
    # Define the file name with the date appended
    report_file_name = f'Weekly_Sanitation_Validation_Report_Failed_{timestamp}.xlsx'
    
    # Save the failed records to an Excel file with the generated file name
    failed_records.to_excel(f'{file_path}\\{report_file_name}', index=False)
else:
    print('Weekly Sanitation Validation Report Passed')  

Weekly Sanitation Validation Report Passed


In [ ]:
#Counts

In [ ]:
 #results_ws_s_c_df.to_csv(file, mode='a', index=False, header=False)

## 3) Weekly Rotation Counts & Data Validation between source and target ##

## 4) 3xWeekly Counts & Data Validation between source and target ##

## 5) Periodic Counts & Data Validation between source and target ##

## 6) Quaterly Counts & Data Validation between source and target ##

In [ ]:
Counts_Report_Date 
 Daily count Source - Done
 Daily count Target - Done
 Weekly Sanitation count Source - 
 Weekly Sanitation count Target -
 Weekly Rotation count Source - 
 Weekly Rotation count Target - 
 3xWeekly count Source - 
 3xWeekly count Target - 
 Periodic count Source - 
 Periodic count Target - 
 
 ------------------------
Daily_Validation_Report_Date - Done
Weekly_Sanitation_Validation_Report_Date - Done
Weekly_Rotation_Validation_Report_Date
3xWeekly_ReportValidation_Report_Date
Periodic_Validation_Report_Date
Quaterly_Validation_Report_Date

 

In [ ]:
#merged_df.to_csv("C:\\Users\\AS81970\\Desktop\\tm.csv", index=False)